In [1]:
from sklearn import datasets
import numpy as np
import random
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
iris = datasets.load_iris()

//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.base import BaseEstimator, ClassifierMixin
class PerceptronClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, max_iteration=1000, lr=0.01):
        self.max_iteration = max_iteration
        self.lr = lr

    def fit(self, X, y=None):
        rows,columns = X.shape
        weight = np.ones(columns)
        bias = 0
        train_data = np.concatenate((X, np.array(y).reshape(rows,1)), axis=1)
        # gradient descent
        for i in range(self.max_iteration):
            for sample in train_data:
                x,y = sample[:-1],sample[-1]
                y = 1 if y == 1 else -1
                predict = np.dot(weight,x.T) + bias
                if y * predict < 0: # 错分样本才更新权重
                    weight = weight - self.lr * x
                    bias = bias - self.lr * 1
        self.weight = weight
        self.bias = bias
        return self
    def predict(self, X, y=None):
        result = []
        for i in range(len(X)):
            prediction = np.dot(self.weight, X[i].T) + self.bias
            if prediction > 0:
                result.append(1)
            else:
                result.append(0)
        return result
    
    def predict_proba(self, X, y=None):
        result = []
        for i in range(len(X)):
            prediction = np.dot(self.weight, X[i].T) + self.bias
            if prediction > 0:
                result.append([0, 1])
            else:
                result.append([1, 0])
        return np.array(result)

In [3]:
from sklearn.base import BaseEstimator, ClassifierMixin
class RelaxationClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, max_iteration=1000, lr=0.01, b=0.1):
        self.max_iteration = max_iteration
        self.b = b
        self.lr = lr

    def fit(self, X, y=None):
        rows, columns = X.shape
        weight_aug = np.ones(X.shape[1] + 1)
        X_aug = np.column_stack((X, np.ones(len(X))))
        # gradient descent
        for _ in range(self.max_iteration):
            for row_id in range(len(X_aug)):
                prediction = np.dot(weight_aug, X_aug[row_id].T) - self.b
                y[row_id] = 1 if y[row_id] == 1 else -1
                if y[row_id] * prediction < 0: # 错分样本才更新权重
                    weight_aug = weight_aug - self.lr * prediction / np.linalg.norm(X_aug[row_id]) * X_aug[row_id].T
        self.weight_aug = weight_aug
        return self
    def predict(self, X, y=None):
        result = []
        X_aug = np.column_stack((X, np.ones(len(X))))
        for row_id in range(len(X)):
            prediction = np.dot(self.weight_aug, X_aug[row_id].T) - self.b
            if prediction > 0:
                result.append(1)
            else:
                result.append(0)
        return result
    
    def predict_proba(self, X, y=None):
        result = []
        X_aug = np.column_stack((X, np.ones(len(X))))
        for row_id in range(len(X)):
            prediction = np.dot(self.weight_aug, X_aug[row_id].T) - self.b
            if prediction > 0:
                result.append([0, 1])
            else:
                result.append([1, 0])
        return np.array(result)

In [4]:
from sklearn.base import BaseEstimator, ClassifierMixin
class MSEClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, threshold=1, lr=0.01):
        self.threshold = threshold
        self.lr = lr

    def fit(self, X, y=None):
        rows, columns = X.shape
        weight_aug = np.ones(X.shape[1] + 1)
        X_aug = np.column_stack((X, np.ones(len(X))))
        for i in range(rows):
            if y[i] == 0: X_aug[i] = -1 * X_aug[i]
        b = np.ones(len(X))
        # gradient descent
        delta = 9999
        k = 0
        while delta > self.threshold:
            row_id = k % len(X)
            delta_vector = self.lr * (np.dot(b[row_id] - np.dot(weight_aug, X_aug[row_id].T), X_aug[row_id].T))
            weight_aug = weight_aug + delta_vector
            delta = np.linalg.norm(delta_vector)
            k += 1
        self.weight_aug = weight_aug
        return self
    def predict(self, X, y=None):
        result = []
        X_aug = np.column_stack((X, np.ones(len(X))))
        for row_id in range(len(X)):
            prediction = np.dot(self.weight_aug, X_aug[row_id].T)
            if prediction > 0:
                result.append(1)
            else:
                result.append(0)
        return result
    
    def predict_proba(self, X, y=None):
        result = []
        X_aug = np.column_stack((X, np.ones(len(X))))
        for row_id in range(len(X)):
            prediction = np.dot(self.weight_aug, X_aug[row_id].T)
            if prediction > 0:
                result.append([0, 1])
            else:
                result.append([1, 0])
        return np.array(result)

In [23]:
from sklearn.base import BaseEstimator, ClassifierMixin
class FisherClassifier(BaseEstimator, ClassifierMixin):      
    def compute_cov_mean(self,x):
        u = np.mean(x, axis=0)
        cov = np.zeros((x.shape[1], x.shape[1]))
        for s in x:
            temp = (s - u).reshape(4, 1)
            cov += np.dot(temp, temp.T)
        return cov,u

    def fit(self, X, y=None):
        num_train,num_features = X.shape
        x_1 = X[np.where(y == 1)]
        x_2 = X[np.where(y == 0)]
        conv1, u1 = self.compute_cov_mean(x_1)
        conv2, u2 = self.compute_cov_mean(x_2)
        SW = conv1 + conv2
        u, s, v = np.linalg.svd(SW)
        sw_inv = np.dot(np.dot(v.T, np.linalg.inv(np.diag(s))), u.T)
        self.W = np.dot(sw_inv,u1-u2)
        w_conv = self.W
        y_space1 = np.dot(x_1, w_conv)
        uy1 = np.mean(y_space1)
        y_space2 = np.dot(x_2, w_conv)
        uy2 = np.mean(y_space2)
        self.b = -1/2*(uy1+uy2)
        return self
    
    def predict(self, X, y=None):
        result = []
        for row_id in range(len(X)):
            prediction = np.dot(self.W, X[row_id].T) + self.b
            if prediction > 0:
                result.append(1)
            else:
                result.append(0)
        return result
    
    def predict_proba(self, X, y=None):
        result = []
        for row_id in range(len(X)):
            prediction = np.dot(self.W, X[row_id].T) + self.b
            if prediction > 0:
                result.append([0, 1])
            else:
                result.append([1, 0])
        return np.array(result)

In [6]:
from sklearn.base import BaseEstimator, ClassifierMixin
class HKClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, max_iteration=1000, lr=0.01):
        self.max_iteration = max_iteration
        self.lr = lr

    def fit(self, X, y=None):
        num_train, num_features = X.shape
        xone = np.ones((num_train, 1))
        x = np.column_stack((xone, X))
        x[np.where(y == 0)] = x[np.where(y == 0)] * -1
        #权重和偏置初始化
        self.W = np.ones((num_features+1))
        Y = x
        Ywn = np.linalg.pinv(Y)
        bias = np.ones((num_train,))
        bmin = 1e-6
        flag = False
        for i in range (self.max_iteration):
            temp = np.array(np.dot(Y,self.W))
            Evector = temp-bias
            Evector_ = 1/2 * np.add(Evector, np.absolute(Evector))
            bias = np.add(2 * self.lr * Evector_, bias)
            self.W = np.dot(Ywn, bias)
            for row in Evector_:
                if (np.absolute(row) <= bmin):
                    # reach bmin. End.
                    flag = True
                    break
            if flag: break
        self.b = self.W[0]
        self.W = self.W[1:]
        return self
    
    def predict(self, X, y=None):
        result = []
        for row_id in range(len(X)):
            prediction = np.dot(self.W, X[row_id].T) + self.b
            if prediction > 0:
                result.append(1)
            else:
                result.append(0)
        return result
    
    def predict_proba(self, X, y=None):
        result = []
        for row_id in range(len(X)):
            prediction = np.dot(self.W, X[row_id].T) + self.b
            if prediction > 0:
                result.append([0, 1])
            else:
                result.append([1, 0])
        return np.array(result)

In [7]:
from scipy.spatial.distance import pdist
import operator
from collections import OrderedDict
from collections import Counter
class KNNClassifier():
    def __init__(self, k=5):
        self.k = 5
    
    def fit(self, X, y):
        self.X = X
        self.y = y
    
    def distance(self, x, y):
        X = np.vstack([x,y])
        return pdist(X)
    
    def predict(self, X, y=None):
        result = []
        for row_id in range(len(X)):
            distances = []
            for index in range(len(self.X)):
                dis = self.distance(self.X[index], X[row_id])[0]
                distances.append([index, dis, self.y[index]])
            distances = sorted(distances, key=lambda x: x[1])
            labels = [distances[i][2] for i in range(self.k)]
            result.append(Counter(labels).most_common(1)[0][0])
        return result

### Evaluation

In [8]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.svm import SVC
x_all = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
y_all = iris['target']
test_ratio = 0.8
x_train, x_test, y_train, y_test = train_test_split(x_all.as_matrix(), y_all, test_size=test_ratio, random_state=2018)

In [24]:
# Perceptron
model = OneVsOneClassifier(PerceptronClassifier())
model.fit(x_train, y_train)
print("Perceptron OVO: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsRestClassifier(PerceptronClassifier())
model.fit(x_train, y_train)
print("Perceptron OVR: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
# Relaxation
model = OneVsOneClassifier(RelaxationClassifier())
model.fit(x_train, y_train)
print("Relaxation OVO: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsRestClassifier(RelaxationClassifier())
model.fit(x_train, y_train)
print("Relaxation OVR: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
# Fisher
model = OneVsOneClassifier(FisherClassifier())
model.fit(x_train, y_train)
print("Fisher OVO: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsRestClassifier(FisherClassifier())
model.fit(x_train, y_train)
print("Fisher OVR: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
# MSE
model = OneVsOneClassifier(MSEClassifier())
model.fit(x_train, y_train)
print("MSE OVO: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsRestClassifier(MSEClassifier())
model.fit(x_train, y_train)
print("MSE OVR: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
# Ho-Kashyap
model = OneVsOneClassifier(HKClassifier())
model.fit(x_train, y_train)
print("Ho-Kashyap OVO: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsRestClassifier(HKClassifier())
model.fit(x_train, y_train)
print("Ho-Kashyap OVR: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
# KNN
model = KNNClassifier(k=5)
model.fit(x_train, y_train)
model.predict(x_test)
print("KNN: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
# SVM
model = OneVsOneClassifier(SVC(kernel='rbf'))
model.fit(x_train, y_train)
print("SVM(rbf) OVO: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsRestClassifier(SVC(kernel='rbf'))
model.fit(x_train, y_train)
print("SVM(rbf) OVR: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsOneClassifier(SVC(kernel='linear'))
model.fit(x_train, y_train)
print("SVM(linear) OVO: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsRestClassifier(SVC(kernel='linear'))
model.fit(x_train, y_train)
print("SVM(linear) OVR: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsOneClassifier(SVC(kernel='poly'))
model.fit(x_train, y_train)
print("SVM(poly) OVO: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsRestClassifier(SVC(kernel='poly'))
model.fit(x_train, y_train)
print("SVM(poly) OVR: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsOneClassifier(SVC(kernel='sigmoid'))
model.fit(x_train, y_train)
print("SVM(sigmoid) OVO: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))
model = OneVsRestClassifier(SVC(kernel='sigmoid'))
model.fit(x_train, y_train)
print("SVM(sigmoid) OVR: {}".format(accuracy_score(y_test, model.predict(x_test), normalize=True)))

Perceptron OVO: 0.6916666666666667
Perceptron OVR: 0.30833333333333335
Relaxation OVO: 0.8833333333333333
Relaxation OVR: 0.675
Fisher OVO: 0.95
Fisher OVR: 0.7333333333333333
MSE OVO: 0.30833333333333335
MSE OVR: 0.30833333333333335
Ho-Kashyap OVO: 0.9583333333333334
Ho-Kashyap OVR: 0.7333333333333333
KNN: 0.9416666666666667
SVM(rbf) OVO: 0.975
SVM(rbf) OVR: 0.9666666666666667
SVM(linear) OVO: 0.9666666666666667
SVM(linear) OVR: 0.775
SVM(poly) OVO: 0.95
SVM(poly) OVR: 0.95
SVM(sigmoid) OVO: 0.30833333333333335
SVM(sigmoid) OVR: 0.30833333333333335
